In [41]:
import os
import libsql
from dotenv import load_dotenv

load_dotenv()

url = os.getenv("CLAUDELIGHT_DB_URL")
auth_token = os.getenv("CLAUDELIGHT_RW")
conn = libsql.connect("db_BZ/claude_light.db", sync_url=url, auth_token=auth_token)
conn.sync()

# Create Table in db
conn.execute('''create table if not exists
measurements (
id INTEGER PRIMARY KEY, -- this is the rowid
data JSON -- JSON blob (stored as TEXT under the hood)
)''')
conn.sync()

## Collect data to start

In [42]:
# Collect data

import time
import requests
import getpass
import socket
import inspect
import hashlib
import ast
import json

def rgb(R=0, G=0, B=0, tag=None):
    """Run Claude-Light.
    R: float
    The red channel setting (0-1)
    G: float
    The green channel setting (0-1)
    B: float
    The blue channel setting (0-1)
    tag: string
    A label for grouping measurements
    Returns
    -------
    A dictionary with metadata and results
    """
    source = inspect.getsource(rgb)
    # Parse and unparse to normalize formatting
    tree = ast.parse(source)
    normalized = ast.unparse(tree)
    d = {'version': hashlib.sha256(normalized.encode()).hexdigest(), 
         'func': normalized,
         't0': time.time(),
         'user': getpass.getuser(),
         'hostname': socket.gethostname(),
         'tag': tag}
    res = requests.get('https://claude-light.cheme.cmu.edu/api', params={'R': R, 'G': G, 'B': B})
    d.update(res.json())
    return d

In [43]:
# Collect one backgroud data with all 0

# res = rgb(tag='background')
# res

In [44]:
# Store data to db

# conn.execute("INSERT INTO measurements(data) VALUES (?);", [json.dumps(res)])
# conn.commit()
# conn.sync()

In [45]:
# Read data from db

# for rowid, data in conn.execute("""select * from measurements where json_extract(data, '$.tag') = 'background'""").fetchall():
#     j = json.loads(data)
#     if rowid == 0 | rowid == 2:
#         print(j['out'])

In [46]:
# j

## Collect more data

In [47]:
today_str = time.strftime("%Y%m%d")
print(today_str)

20251101


In [48]:
# Collect more data
import random

for i in range(25):
    if i == 0:
        r = 0.5
        g = 0.5
        b = 0.5
        res = rgb(r, g, b, tag=today_str)
    else:
        r = round(random.random(), 1)
        g = round(random.random(), 1)
        b = round(random.random(), 1)
        res = rgb(r, g, b, tag=today_str)
    # Store to db
    conn.execute("INSERT INTO measurements(data) VALUES (?);", [json.dumps(res)])
    conn.commit()
    conn.sync()
    
    time.sleep(3)

In [49]:
# r = 0.5
# g = 0.5
# b = 0.5
# res = rgb(r, g, b, tag=today_str)
# # Store to db
# conn.execute("INSERT INTO measurements(data) VALUES (?);", [json.dumps(res)])
# conn.commit()
# conn.sync()

In [50]:
# for rowid, data in conn.execute(f"""SELECT * FROM measurements WHERE json_extract(data, '$.tag') = '{today_str}'""").fetchall():
#     j = json.loads(data)
#     print(j)